In [ ]:
pip install langchain langchain_openai langchain_community chromadb

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [7]:

# 쿼리를 위한 로그 설정
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)


In [2]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로더 설정
loaders = [TextLoader("./Data/How_to_invest_money.txt")]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [4]:
# 문서 생성을 위한 텍스트 분할기 정의
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 문서 분할
split_docs = recursive_splitter.split_documents(docs)

# OpenAIEmbeddings 인스턴스 생성
embeddings = OpenAIEmbeddings()

# Chroma vectorstore 생성
vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)

In [5]:
from langchain.retrievers import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 모델 설정 (여기서는 ChatOpenAI 사용)
llm = ChatOpenAI(model = "gpt-4o",temperature=0.2)

# MultiQueryRetriever 실행
retriever = MultiQueryRetriever.from_llm(
retriever=vectorstore.as_retriever(), # 기본 검색기 (벡터 데이터베이스)
llm=llm, # 앞서 정의한 llm (gpt-4o)
)

In [8]:
# 샘플 질문
question = "주식 투자를 처음 시작하려면 어떻게 해야 하나요?"

# 결과 검색
unique_docs = retriever.invoke(question)
print(f"\n결과: {len(unique_docs)}개의 문서가 검색되었습니다.")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 주식 투자 초보자가 알아야 할 기본 단계는 무엇인가요?', '2. 주식 투자를 처음 시작할 때 필요한 준비 사항은 무엇인가요?', '3. 주식 투자 입문자를 위한 가이드라인이나 조언이 있나요?']



결과: 6개의 문서가 검색되었습니다.


In [10]:
unique_docs

[Document(metadata={'source': './Data/How_to_invest_money.txt'}, page_content='G. G. H.\n\n\n\n\nHOW TO INVEST MONEY\n\n\n\n\nI\n\nGENERAL PRINCIPLES OF INVESTMENT'),
 Document(metadata={'source': './Data/How_to_invest_money.txt'}, page_content='For the successful investment of money, however, a good deal more is\nrequired than the mere ability to select a safe security. That is only\none phase of the problem. Scientific investment demands a clear\nunderstanding of the fundamental distinctions between different classes\nof securities and strict adherence to the two cardinal principles,\ndistribution of risk and selection of securities in accordance with\nreal requirements.'),
 Document(metadata={'source': './Data/How_to_invest_money.txt'}, page_content='After learning how to judge the value of every form of investment, a man\nmay still be unsuccessful in the investment of money unless he acquires\nalso a firm grasp upon the general principles which control the price\nmovements of secur

In [9]:
from langchain.chains import RetrievalQA

# RetrievalQA 체인 설정
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 질문에 대한 답변 생성
result = qa_chain.invoke({"query": question})

# 결과 출력
print("답변:", result["result"])
print("\n사용된 문서:")
for doc in result["source_documents"]:
    print(doc.page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 주식 투자 초보자가 알아야 할 기본 단계는 무엇인가요?', '2. 주식 투자를 처음 시작할 때 필요한 준비 사항은 무엇인가요?', '3. 주식 투자 입문자를 위한 가이드라인이나 조언이 있나요?']


답변: 주식 투자를 처음 시작하려면 몇 가지 기본적인 단계를 따르는 것이 중요합니다:

1. **목표 설정**: 투자 목표를 명확히 하세요. 예를 들어, 장기적인 자산 증식, 단기적인 수익 창출, 은퇴 자금 마련 등 구체적인 목표를 설정하세요.

2. **기본 지식 습득**: 주식 시장의 기본 개념과 작동 방식을 이해하세요. 주식과 채권의 차이점, 주식 시장의 작동 원리 등을 배우는 것이 중요합니다.

3. **재정 상태 평가**: 자신의 재정 상태를 평가하고, 투자 가능한 금액을 결정하세요. 이는 투자 리스크를 관리하는 데 중요합니다.

4. **투자 전략 수립**: 자신의 투자 성향에 맞는 전략을 수립하세요. 예를 들어, 장기 투자, 단기 매매, 배당주 투자 등 다양한 전략이 있습니다.

5. **증권 계좌 개설**: 주식을 거래하기 위해 증권 계좌를 개설하세요. 신뢰할 수 있는 증권사를 선택하는 것이 중요합니다.

6. **시장 조사 및 분석**: 투자할 기업이나 산업에 대해 철저히 조사하고 분석하세요. 이는 정보에 기반한 결정을 내리는 데 도움을 줍니다.

7. **전문가 상담**: 필요하다면 투자 전문가나 금융 고문과 상담하여 조언을 구하세요. 이는 더 나은 투자 결정을 내리는 데 도움이 될 수 있습니다.

8. **소액으로 시작**: 처음에는 소액으로 시작하여 경험을 쌓고, 점차 투자 규모를 늘려가는 것이 좋습니다.

이러한 단계를 통해 주식 투자를 시작할 수 있으며, 지속적으로 학습하고 시장 변화를 주시하는 것이 중요합니다.

사용된 문서:
G. G. H.




HOW TO INVEST MONEY




I

GENERAL PRINCIPLES OF INVESTMENT
For the successful investment of money, however, a good deal more is
required than the mere ability to select a safe security. That is only
one phase of the p

In [11]:
result

{'query': '주식 투자를 처음 시작하려면 어떻게 해야 하나요?',
 'result': '주식 투자를 처음 시작하려면 몇 가지 기본적인 단계를 따르는 것이 중요합니다:\n\n1. **목표 설정**: 투자 목표를 명확히 하세요. 예를 들어, 장기적인 자산 증식, 단기적인 수익 창출, 은퇴 자금 마련 등 구체적인 목표를 설정하세요.\n\n2. **기본 지식 습득**: 주식 시장의 기본 개념과 작동 방식을 이해하세요. 주식과 채권의 차이점, 주식 시장의 작동 원리 등을 배우는 것이 중요합니다.\n\n3. **재정 상태 평가**: 자신의 재정 상태를 평가하고, 투자 가능한 금액을 결정하세요. 이는 투자 리스크를 관리하는 데 중요합니다.\n\n4. **투자 전략 수립**: 자신의 투자 성향에 맞는 전략을 수립하세요. 예를 들어, 장기 투자, 단기 매매, 배당주 투자 등 다양한 전략이 있습니다.\n\n5. **증권 계좌 개설**: 주식을 거래하기 위해 증권 계좌를 개설하세요. 신뢰할 수 있는 증권사를 선택하는 것이 중요합니다.\n\n6. **시장 조사 및 분석**: 투자할 기업이나 산업에 대해 철저히 조사하고 분석하세요. 이는 정보에 기반한 결정을 내리는 데 도움을 줍니다.\n\n7. **전문가 상담**: 필요하다면 투자 전문가나 금융 고문과 상담하여 조언을 구하세요. 이는 더 나은 투자 결정을 내리는 데 도움이 될 수 있습니다.\n\n8. **소액으로 시작**: 처음에는 소액으로 시작하여 경험을 쌓고, 점차 투자 규모를 늘려가는 것이 좋습니다.\n\n이러한 단계를 통해 주식 투자를 시작할 수 있으며, 지속적으로 학습하고 시장 변화를 주시하는 것이 중요합니다.',
 'source_documents': [Document(metadata={'source': './Data/How_to_invest_money.txt'}, page_content='G. G. H.\n\n\n\n\nHOW TO INVEST MONEY\n\n\n\n\nI\n\nGENERAL PRINCIPLES 